---
sidebar_label: Github
---

# GithubToolkit

The `Github` toolkit contains tools that enable an LLM agent to interact with a github repository. 
The tool is a wrapper for the [PyGitHub](https://github.com/PyGithub/PyGithub) library. 

For detailed documentation of all GithubToolkit features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.github.toolkit.GitHubToolkit.html).

## Setup

At a high-level, we will:

1. Install the pygithub library
2. Create a Github app
3. Set your environmental variables
4. Pass the tools to your agent with `toolkit.get_tools()`

If you want to get automated tracing from runs of individual tools, you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

#### 1. Install dependencies

This integration is implemented in `langchain-community`. We will also need the `pygithub` dependency:

In [ ]:
%pip install --upgrade --quiet  pygithub langchain-community

#### 2. Create a Github App

[Follow the instructions here](https://docs.github.com/en/apps/creating-github-apps/registering-a-github-app/registering-a-github-app) to create and register a Github app. Make sure your app has the following [repository permissions:](https://docs.github.com/en/rest/overview/permissions-required-for-github-apps?apiVersion=2022-11-28)

* Commit statuses (read only)
* Contents (read and write)
* Issues (read and write)
* Metadata (read only)
* Pull requests (read and write)

Once the app has been registered, you must give your app permission to access each of the repositories you whish it to act upon. Use the App settings on [github.com here](https://github.com/settings/installations).


#### 3. Set Environment Variables

Before initializing your agent, the following environment variables need to be set:

* **GITHUB_APP_ID**- A six digit number found in your app's general settings
* **GITHUB_APP_PRIVATE_KEY**- The location of your app's private key .pem file, or the full text of that file as a string.
* **GITHUB_REPOSITORY**- The name of the Github repository you want your bot to act upon. Must follow the format {username}/{repo-name}. *Make sure the app has been added to this repository first!*
* Optional: **GITHUB_BRANCH**- The branch where the bot will make its commits. Defaults to `repo.default_branch`.
* Optional: **GITHUB_BASE_BRANCH**- The base branch of your repo upon which PRs will based from. Defaults to `repo.default_branch`.

In [ ]:
import getpass
import os

for env_var in [
    "GITHUB_APP_ID",
    "GITHUB_APP_PRIVATE_KEY",
    "GITHUB_REPOSITORY",
]:
    if not os.getenv(env_var):
        os.environ[env_var] = getpass.getpass()

## Instantiation

Now we can instantiate our toolkit:

In [2]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper

github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)

## Tools

View available tools:

In [3]:
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name)

Get Issues
Get Issue
Comment on Issue
List open pull requests (PRs)
Get Pull Request
Overview of files included in PR
Create Pull Request
List Pull Requests' Files
Create File
Read File
Update File
Delete File
Overview of existing files in Main branch
Overview of files in current working branch
List branches in this repository
Set active branch
Create a new branch
Get files from a directory
Search issues and pull requests
Search code
Create review request


The purpose of these tools is as follows:

Each of these steps will be explained in great detail below.

1. **Get Issues**- fetches issues from the repository.

2. **Get Issue**- fetches details about a specific issue.

3. **Comment on Issue**- posts a comment on a specific issue.

4. **Create Pull Request**- creates a pull request from the bot's working branch to the base branch.

5. **Create File**- creates a new file in the repository.

6. **Read File**- reads a file from the repository.

7. **Update File**- updates a file in the repository.

8. **Delete File**- deletes a file from the repository.

## Use within an agent

We will need a LLM or chat model:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

Initialize the agent with a subset of tools:

In [6]:
from langgraph.prebuilt import create_react_agent

tools = [tool for tool in toolkit.get_tools() if tool.name == "Get Issue"]
assert len(tools) == 1
tools[0].name = "get_issue"

agent_executor = create_react_agent(llm, tools)

And issue it a query:

In [7]:
example_query = "What is the title of issue 24888?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the title of issue 24888?
================================== Ai Message ==================================
Tool Calls:
  get_issue (call_iSYJVaM7uchfNHOMJoVPQsOi)
 Call ID: call_iSYJVaM7uchfNHOMJoVPQsOi
  Args:
    issue_number: 24888
================================= Tool Message =================================
Name: get_issue

{"number": 24888, "title": "Standardize KV-Store Docs", "body": "To make our KV-store integrations as easy to use as possible we need to make sure the docs for them are thorough and standardized. There are two parts to this: updating the KV-store docstrings and updating the actual integration docs.\r\n\r\nThis needs to be done for each KV-store integration, ideally with one PR per KV-store.\r\n\r\nRelated to broader issues #21983 and #22005.\r\n\r\n## Docstrings\r\nEach KV-store class docstring should have the sections shown in the [Appendix](#appendix) below. The sectio

## API reference

For detailed documentation of all `GithubToolkit` features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.github.toolkit.GitHubToolkit.html).